In [1]:
import torch
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer, EsmForMaskedLM
from tokenizers import Tokenizer
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pickle

/opt/anaconda3/envs/MachLearn/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Here we add wild type to protein dictionary.


In [3]:
from plm_compare_esm import *
from protein_data import *

In [4]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
# print(f"Using {device} device")

# model_name = "hugohrban/progen2-medium"
# model, tokenizer = initialize_progen2(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

model_name = "facebook/esm2_t33_650M_UR50D"
model, tokenizer = initialize_esm2(model_name)

Using cpu device


In [5]:
df_otc_h = pd.read_csv('/Users/johnhutchens/Desktop/Practicum/Data/DMS_ProteinGym_substitutions/OTC_HUMAN_Lo_2023.csv')

In [6]:
df_otc_h.head()

,mutant,mutated_sequence,DMS_score,DMS_score_bin
0,A102E,MLFNLRILLNNAAFRNGHNFMVRNFRCGQPLQNKVQLKGRDLLTLK...,0.438,1
1,A102G,MLFNLRILLNNAAFRNGHNFMVRNFRCGQPLQNKVQLKGRDLLTLK...,0.676,1
2,A102P,MLFNLRILLNNAAFRNGHNFMVRNFRCGQPLQNKVQLKGRDLLTLK...,0.153,0
3,A102S,MLFNLRILLNNAAFRNGHNFMVRNFRCGQPLQNKVQLKGRDLLTLK...,0.948,1
4,A102T,MLFNLRILLNNAAFRNGHNFMVRNFRCGQPLQNKVQLKGRDLLTLK...,0.861,1


In [7]:
mut = df_otc_h.iloc[0]['mutant']
seq = df_otc_h.iloc[0]['mutated_sequence']
print(mut)
print(seq)

A102E
MLFNLRILLNNAAFRNGHNFMVRNFRCGQPLQNKVQLKGRDLLTLKNFTGEEIKYMLWLSADLKFRIKQKGEYLPLLQGKSLGMIFEKRSTRTRLSTETGFELLGGHPCFLTTQDIHLGVNESLTDTARVLSSMADAVLARVYKQSDLDTLAKEASIPIINGLSDLYHPIQILADYLTLQEHYSSLKGLTLSWIGDGNNILHSIMMSAAKFGMHLQAATPKGYEPDASVTKLAEQYAKENGTKLLLTNDPLEAAHGGNVLITDTWISMGQEEEKKKRLQAFQGYQVTMKTAKVAASDWTFLHCLPRKPEEVDDEVFYSPRSLVFPEAENRKWTIMAVMVSLLTDYSPQLQKPKF


In [8]:
len_mut = len(mut)
orig = mut[0]
pos = int(mut[1:len_mut-1])-1
new = mut[len_mut-1]

In [9]:
wild_seq = seq[:pos] + orig + seq[pos+1:]

In [10]:
print(seq[101])
wild_seq[101]

E


'A'

In [28]:
with open('/Users/johnhutchens/Desktop/Practicum/Data/OTC_Human/esm_otc_hum_matrices.pickle',
           'rb') as f:
    esm_dict = pickle.load(f)

In [29]:
esm_dict[None]

{'mutated_sequence': 'MLFNLRILLNNAAFRNGHNFMVRNFRCGQPLQNKVQLKGRDLLTLKNFTGEEIKYMLWLSADLKFRIKQKGEYLPLLQGKSLGMIFEKRSTRTRLSTETGFALLGGHPCFLTTQDIHLGVNESLTDTARVLSSMADAVLARVYKQSDLDTLAKEASIPIINGLSDLYHPIQILADYLTLQEHYSSLKGLTLSWIGDGNNILHSIMMSAAKFGMHLQAATPKGYEPDASVTKLAEQYAKENGTKLLLTNDPLEAAHGGNVLITDTWISMGQEEEKKKRLQAFQGYQVTMKTAKVAASDWTFLHCLPRKPEEVDDEVFYSPRSLVFPEAENRKWTIMAVMVSLLTDYSPQLQKPKF',
 'log_probs': tensor([[-12.1060, -13.6520, -11.7604,  ..., -12.3295, -14.5514, -12.4707],
         [ -4.2916,  -5.6300,  -4.8426,  ...,  -4.1973,  -6.7002,  -5.2664],
         [ -2.5552,  -4.3235,  -4.0442,  ...,  -3.4860,  -5.2507,  -4.1625],
         ...,
         [ -3.3189,  -4.7702,  -3.5415,  ...,  -3.2073,  -4.9429,  -3.6887],
         [ -3.0121,  -5.1392,  -3.9973,  ...,  -3.5407,  -5.2016,  -4.2208],
         [ -3.2881,  -4.7278,  -3.7291,  ...,  -3.0776,  -4.8701,  -3.6024]]),
 'ref_log_probs': tensor([[-1.7140e-01],
         [-2.0410e+01],
         [-2.3868e+00],
         [-1.1586e+00],
         [-1.9101

In [21]:
esm_dict[None] = {'mutated_sequence': wild_seq}

In [23]:
sequence = esm_dict[None]['mutated_sequence']
lp, rlp, llr = collect_log_prob_pg2(sequence, model, tokenizer)

esm_dict[None]['log_probs'] = lp
esm_dict[None]['ref_log_probs'] = rlp
esm_dict[None]['llr_matrix'] = llr

In [24]:
esm_dict[None]

{'mutated_sequence': 'MLFNLRILLNNAAFRNGHNFMVRNFRCGQPLQNKVQLKGRDLLTLKNFTGEEIKYMLWLSADLKFRIKQKGEYLPLLQGKSLGMIFEKRSTRTRLSTETGFALLGGHPCFLTTQDIHLGVNESLTDTARVLSSMADAVLARVYKQSDLDTLAKEASIPIINGLSDLYHPIQILADYLTLQEHYSSLKGLTLSWIGDGNNILHSIMMSAAKFGMHLQAATPKGYEPDASVTKLAEQYAKENGTKLLLTNDPLEAAHGGNVLITDTWISMGQEEEKKKRLQAFQGYQVTMKTAKVAASDWTFLHCLPRKPEEVDDEVFYSPRSLVFPEAENRKWTIMAVMVSLLTDYSPQLQKPKF',
 'log_probs': tensor([[-12.1060, -13.6520, -11.7604,  ..., -12.3295, -14.5514, -12.4707],
         [ -4.2916,  -5.6300,  -4.8426,  ...,  -4.1973,  -6.7002,  -5.2664],
         [ -2.5552,  -4.3235,  -4.0442,  ...,  -3.4860,  -5.2507,  -4.1625],
         ...,
         [ -3.3189,  -4.7702,  -3.5415,  ...,  -3.2073,  -4.9429,  -3.6887],
         [ -3.0121,  -5.1392,  -3.9973,  ...,  -3.5407,  -5.2016,  -4.2208],
         [ -3.2881,  -4.7278,  -3.7291,  ...,  -3.0776,  -4.8701,  -3.6024]]),
 'ref_log_probs': tensor([[-1.7140e-01],
         [-2.0410e+01],
         [-2.3868e+00],
         [-1.1586e+00],
         [-1.9101

In [25]:
filename = '/Users/johnhutchens/Desktop/Practicum/Data/OTC_Human/esm_otc_hum_matrices.pickle' 

with open(filename, 'wb') as f:
    pickle.dump(esm_dict, f, protocol=pickle.HIGHEST_PROTOCOL)
